In [1]:
# %load /cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func/base_imports.py
import pandas as pd
import numpy as np
import tqdm 
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tarfile
import re
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
import sys
sys.path.append(r"/cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func")
from config import *
from functions import *

In [3]:
# Loading tables according to given paths

filter_dir = os.path.join(FILTERING_PATH_BRCA, 'filtering_samples', FILTERING_ID)

out_df_filtered=pd.DataFrame()

out_var=''
title_venn = '{sample}'

# ETH Names
# OLD code
if not OHSU_BRCA_NEW: 
    eth_all = glob.glob(os.path.join(filter_dir, 'G*'))
    salt=''
    file=TAR_OHSU_BRCA
    
#New code
if OHSU_BRCA_NEW:
    eth_all = glob.glob(os.path.join(filter_dir, 'G*'))
    eth_all = [i for i in eth_all if len(os.path.basename(i)) == 45] #Shorten path has len 45
    salt='OHSU_BRSA_NEW'
    file=TAR_OSHU_BRCA_NEW
    
# print(eth_all,TT)
# OHSU Names
with tarfile.open(file, "r:*") as tar:
    ohsu_all = tar.getnames()

print(ohsu_all[0],TT)
# Get file pairs
file_pair = {}
for idx_eth, eth in enumerate(eth_all):
    pattern = os.path.basename(eth).replace('G_', '').replace('.gz', '') 
    for idx_ohsu, ohsu in enumerate(ohsu_all):
        if pattern in ohsu:
            file_pair[eth] = ohsu
    
# print(file_pair,TT)
        
restricts = RESTRICTS_BRCA
for restrict in restricts:
    df = {'sample' : [], 
      'filter_foreground' : [], 
      'filter_background' : [], 
      'filter': [],
      'size_ohsu' : [], 
      'size_eth' : [], 
      'size_intersection' : [], 
      'size_ohsu\eth' : [], 
      'size_eth\ohsu' : [],
      'eth_kmers\inter':[],
      'ohsu_kmers\inter':[],
      'inter_kmers':[],
          
      'coord_OHSU':[],
      'coord_ETH':[],
      'size_ohsu_coor' : [], 
      'size_eth_coor' : [], 
      'size_intersection_coor' : [], 
      'size_ohsu\eth_coor' : [], 
      'size_eth\ohsu_coor' : [],
      'eth_coor\inter_coor':[],
      'ohsu_coor\inter_coor':[],
      'inter_coor':[],
      'eth_coor\ohsu_coor':[],
      'ohsu_coor\eth_coor':[],
      'priority':[],
         }
    with tarfile.open(file, "r:*") as tar: #OHSU
        for eth, ohsu in file_pair.items(): # ETH
            print('eth',eth,TT,'ohsu',ohsu)
            if (not restrict) or restrict == re.findall('G_([\s\S]+?)_',eth)[0].replace('-',''): #Restrict to category of interest
                    print(restrict)
                    df_ohsu = pd.read_csv(ohsu_path:=tar.extractfile(ohsu), sep="\t")
                    df_ohsu.reset_index(inplace=True)
                    if not df_ohsu.empty: df_ohsu = table_processing.ohsu_to_eth_coord(df_ohsu)
                    if not df_ohsu.empty: df_ohsu['junction_coordinate'] = df_ohsu['jx_shifted'].apply(lambda x: ':'.join(x.split(';')[1:3]))
                    df_eth = pd.read_csv(eth, sep="\t")
                    if not df_eth.empty: df_eth=table_processing.get_junction_coordinates(df_eth,'coord')
                    df1=df_eth
                    df2=df_ohsu
                    df_eth = set(df_eth['kmer'])
                    df_ohsu = set(df_ohsu['index'])
                    df_eth_coor = set(df1['junction_coordinate']) if not df1.empty else set([])
                    df_ohsu_coor = set(df2['junction_coordinate']) if not df1.empty else set([])
                    name = os.path.basename(ohsu).replace('.tsv', '').split('_')
                    print(name)
                    df['coord_ETH'].append(df1['junction_coordinate'] if not df1.empty else 'None' )
                    df['coord_OHSU'].append(df2['junction_coordinate'] if not df2.empty else 'None')
                    df['sample'].append(ss:=name[1].replace('-',''))
                
                    if not OHSU_BRCA_NEW:
                        df['filter_foreground'].append(name[2])
                        df['filter_background'].append(name[3])
                        if name[3][1] == 'Any':
                            priority=0
                        elif name[3][1] == 10:
                            priority=1
                        elif name[3][1] == 2:
                            priority=2
                        else:
                            priotity = None
                        df['priority'].append(priority)
                        df['filter'].append(name[2]+' '+name[3])
                    else:
                        df['filter'].append(name[2])
                        a = []
                        for i in range(5):
                            if name[2][i] == 'A':
                                a.append('Any')
                            elif name[2][i] == 'X':
                                a.append('10')
                            elif name[2][i] == 'N':
                                a.append('None')
                            else:
                                a.append(name[2][i])
                        df['filter_foreground'].append(f'({a[0]}, {a[1]}, {a[2]})')
                        df['filter_background'].append(f'({a[3]}, {a[4]})')
                        if a[4] == 'Any':
                            priority=0
                        elif a[4] == '10':
                            priority=1
                        elif a[4] == '2':
                            priority=2
                        else:
                            priority = None
                        df['priority'].append(priority)
                    print(a)
                    print(priority)
                    df['size_ohsu'].append(len(df_ohsu))
                    df['size_eth'].append(len(df_eth))
                    df['size_ohsu_coor'].append(len(df_ohsu_coor))
                    df['size_eth_coor'].append(len(df_eth_coor))
                    df['size_ohsu\eth'].append(len(df_ohsu_filter:=df_ohsu.difference(df_eth)))
                    df['size_eth\ohsu'].append(len(df_eth_filter:=df_eth.difference(df_ohsu)))
                
                    if name[2] in ['0AN3XGA','0AN3AGA'] and name[1].replace('-','') == 'TCGAC8A12P01A11RA11507':
                       out_var+= f'sample: {ss}\n filter_for: ({a[0]}, {a[1]}, {a[2]})\n  filter_back: ({a[3]}, {a[4]})\n ohsu/eth: {len(df_ohsu_filter)}\n path_OSHU: {ohsu_path}\n path_ETH: {eth}\n'
                    
                    df['size_ohsu\eth_coor'].append(len(df_ohsu_filter_coor:=df_ohsu_coor.difference(df_eth_coor)))
                    df['size_eth\ohsu_coor'].append(len(df_eth_filter_coor:=df_eth_coor.difference(df_ohsu_coor)))
                    df['ohsu_coor\eth_coor'].append(np.array(df_ohsu_filter_coor))
                    print(len(df_ohsu_filter_coor))
                    print(len(df_eth_filter_coor))
                    df['eth_coor\ohsu_coor'].append(np.array(df_eth_filter_coor))
                    df['size_intersection'].append(len(df_inter_filter:=set(df_ohsu) & set(df_eth)))
                    df['size_intersection_coor'].append(len(df_inter_filter_coor:=df_ohsu_coor & df_eth_coor))
                    print(len(df_inter_filter_coor))
                    print('NEXT')
                    df['eth_kmers\inter'].append(df_eth_witout_inter:=list(df_eth_filter.difference(df_inter_filter)))
                    df['ohsu_kmers\inter'].append(df_ohsu_witout_inter:=list(df_ohsu_filter.difference(df_inter_filter)))
                    df['eth_coor\inter_coor'].append(df_eth_witout_inter_coor:=list(df_eth_filter_coor.difference(df_inter_filter_coor)))
                    df['ohsu_coor\inter_coor'].append(df_ohsu_witout_inter_coor:=list(df_ohsu_filter_coor.difference(df_inter_filter_coor)))
                    df['inter_kmers'].append(list(df_inter_filter))
                    df['inter_coor'].append(list(df_inter_filter_coor))
    df = pd.DataFrame(df)
         #print(df)
    if not out_df_filtered.empty:
        out_df_filtered = pd.concat([out_df_filtered, df])
    else:
        out_df_filtered = df

J_TCGA-24-1431-01A-01R-1566-13_02101GA.tsv 
---------------------------------------------

eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-A2-A0D2-01A-21R-A034-07_02501GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-A2-A0D2-01A-21R-A034-07_02501GA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-C8-A12P-01A-11R-A115-07_0A512GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-C8-A12P-01A-11R-A115-07_0A512GA.tsv
TCGAC8A12P01A11RA11507


46it [00:00, 6555.83it/s]


['J', 'TCGA-C8-A12P-01A-11R-A115-07', '0A512GA']
['0', 'Any', '5', '1', '2']
2
11
1
3
NEXT
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-A2-A0D2-01A-21R-A034-07_0A501GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-A2-A0D2-01A-21R-A034-07_0A501GA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-AO-A0JM-01A-21R-A056-07_0A11XGA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-AO-A0JM-01A-21R-A056-07_0A11XGA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-A2-A0D2-01A-21R-A034-07_0251AGA.tsv.gz 
--

207it [00:00, 6472.50it/s]


['J', 'TCGA-C8-A12P-01A-11R-A115-07', '0AN1AGA']
['0', 'Any', 'None', '1', 'Any']
0
922
5
20
NEXT
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-C8-A12P-01A-11R-A115-07_0A53AGA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-C8-A12P-01A-11R-A115-07_0A53AGA.tsv
TCGAC8A12P01A11RA11507


...

['J', 'TCGA-A2-A0SX-01A-12R-A084-07', '0A51AGA']
['0', 'Any', '5', '1', 'Any']
0
10
2
3
NEXT
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-C8-A12P-01A-11R-A115-07_0A11AGA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-C8-A12P-01A-11R-A115-07_0A11AGA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-A2-A0D2-01A-21R-A034-07_0A13XGA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-A2-A0D2-01A-21R-A034-07_0A13XGA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-C8-A12P-01A-11R-A115-07_0251AGA.tsv.gz 


180it [00:00, 6604.68it/s]


['J', 'TCGA-A2-A0SX-01A-12R-A084-07', '0A132GA']
['0', 'Any', '1', '3', '2']
2
94
2
12
NEXT
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-A2-A0D2-01A-21R-A034-07_0A112GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-A2-A0D2-01A-21R-A034-07_0A112GA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-C8-A12P-01A-11R-A115-07_02532GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-C8-A12P-01A-11R-A115-07_02532GA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-AO-A0JM-01A-21R-A056-07_0A53AGA.tsv.gz 
-

447it [00:00, 6517.31it/s]

['J', 'TCGA-A2-A0SX-01A-12R-A084-07', '0AN1XGA']
['0', 'Any', 'None', '1', '10']
1
526
6
26
NEXT


END !!!

In [ ]:
out_df_original = pd.read_csv(f'{SAVE_DIR}/{DIR_CSV}/{DIR_BRCA}/{NAME_TABLES}/{NAME_NON_FILTERING_BRCA}', sep="\t")
out_df_original

In [ ]:
task_df={
    'sample':[],
    'filter':[],
    'kmer_eth/inter (filtered)':[],
    'kmer_eth (before filtering)':[],
    'kmer_inter (before filtering)':[],
}

for index, row in out_df_filtered.iterrows():
    for kmer in row['eth_kmers\inter']:
        task_df['sample'].append(sample:=row['sample'])
        if not OHSU_BRCA_NEW:
            task_df['filter'].append(row['filter_foreground']+'  '+row['filter_background'])
        if OHSU_BRCA_NEW:
            task_df['filter'].append(row['filter'])
        task_df['kmer_eth/inter (filtered)'].append(kmer)
        udo_inter=list(out_df_original['kmer_inter'].loc[(out_df_original['sample']==sample)])
        udo_eth=list(out_df_original['kmer_eth'].loc[(out_df_original['sample']==sample)])
        task_df['kmer_eth (before filtering)'].append(1) if kmer in udo_eth[0] else task_df['kmer_eth (before filtering)'].append(0)
        task_df['kmer_inter (before filtering)'].append(1) if kmer in udo_inter[0] else task_df['kmer_inter (before filtering)'].append(0)

df_eth_task=pd.DataFrame(task_df)

In [ ]:
task_ohsu_df={
    'sample':[],
    'filter':[],
    'kmer_ohsu/inter (filtered)':[],
    'kmer_ohsu (before filtering)':[],
    'kmer_inter (before filtering)':[],
}

for index, row in out_df_filtered.iterrows():
    for kmer in row['ohsu_kmers\inter']:
        task_ohsu_df['sample'].append(sample:=row['sample'])
        if not OHSU_BRCA_NEW:
            task_ohsu_df['filter'].append(row['filter_foreground']+'  '+row['filter_background'])
        if OHSU_BRCA_NEW:
            task_ohsu_df['filter'].append(row['filter'])
        task_ohsu_df['kmer_ohsu/inter (filtered)'].append(kmer)
        udo_inter=list(out_df_original['kmer_inter'].loc[(out_df_original['sample']==sample)])[0]
        udo_ohsu=list(out_df_original['kmer_ohsu'].loc[(out_df_original['sample']==sample)])[0]
        task_ohsu_df['kmer_ohsu (before filtering)'].append(1) if kmer in udo_ohsu else task_ohsu_df['kmer_ohsu (before filtering)'].append(0)
        task_ohsu_df['kmer_inter (before filtering)'].append(1) if kmer in udo_inter else task_ohsu_df['kmer_inter (before filtering)'].append(0)

df_ohsu_task=pd.DataFrame(task_ohsu_df)

In [3]:
if not OHSU_BRCA_NEW:
    PATH_DATA = create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_BRCA,NAME_TABLES])
if OHSU_BRCA_NEW:
    PATH_DATA = create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_BRCA,NAME_TABLES,'OHSU_BRCA_NEW',''])
path_filtering=create_path.create_path(PATH_DATA,[NAME_FILTERING_BRCA])
path_ETH=create_path.create_path(PATH_DATA,[NAME_ETH_TASK_BRCA])
path_OHSU=create_path.create_path(PATH_DATA,[NAME_OHSU_TASK_BRCA])

out_df_filtered.to_csv(path_filtering,header=True,sep=';')
#df_eth_task.to_csv(path_ETH,header=True,sep=';')
#df_ohsu_task.to_csv(path_OHSU,header=True,sep=';')